In [ ]:
import pandas as pd

train_data = pd.read_csv('data/train_copy.csv', sep=',')
train_df = train_data.copy()

# Выстраиваем действия пользователя в хронологическом порядке
train_df.sort_values(by=['user_id', 'timestamp'])

# Удаляем столбцы, которые не несут для нас полезной информации
train_df.drop(['row_id', 'timestamp', 'user_answer'], axis=1, inplace=True)

# Удаляем столбцы, с  которыми не знаем что делать
train_df.drop(['task_container_id'], axis=1, inplace=True)

# Количество уникальных пользователей
users = train_df['user_id'].nunique()

# print(train_df)

# Далее будем анализировать тесты 
# Отфильтруем не тесты
mask_question_event = train_df['content_type_id'] == 0.0
# Тесты в порядке возрастания сложности
train_df[mask_question_event].groupby(by='content_id')['answered_correctly'].mean().sort_values(ascending=False)
print(train_df[mask_question_event].groupby(by='content_id')['answered_correctly'].mean().sort_values(ascending=False))

# Гробовые тесты, на которые пытались ответить больше 20% пользователей, но правильно ответило меньше 5%
hardest_test = train_df[mask_question_event].groupby(by='content_id')['answered_correctly'].agg(['count', 'mean']).sort_values(by=['mean'], ascending=False)
mask = (hardest_test['mean'] < 0.05) & (hardest_test['count'] > users/5)
print(hardest_test[mask])

# Для вычисления затраченного на один вопрос теста времени нужно сделать преобразование
# ???

# Анализируем работу пользователей
# Рейтинг пользователей с наибольшим количеством правильных ответов
print(train_df[mask_question_event].groupby(by='user_id')['answered_correctly'].count().sort_values(ascending=False))

# Со временем не будем работать - нет времени! =В

questions_data = pd.read_csv('data/questions.csv', sep=',')
questions_df = questions_data.copy()

print(questions_df.head())

questions = questions_df['question_id'].nunique()
parts = questions_df['part'].nunique()
tags = questions_df['tags'].nunique()

print(questions, parts, tags)

# lectures_data = pd.read_csv('data/questions.csv', sep=',')
# lectures_df = lectures_data.copy()

# lectures = lectures_df['lecture_id'].nunique()

# Сводная таблица по пользователям
# Склеим 2 таблицы
merged = train_df[mask_question_event].merge(
    questions_df,
    right_on='question_id',
    left_on='content_id',
    how='outer'
)

print(merged.tail())

# Оценим знания разделов по каждому пользователю
pivot = merged.pivot_table(
    values='answered_correctly',
    index='user_id',
    columns='part',
    aggfunc=['mean'],
    fill_value=0
)

print(pivot.tail())

# Оценим самые сложные раздел
print(merged.groupby(by='part')['answered_correctly'].mean().sort_values(ascending=False))